### A.) Create RLmlp so that 10 states are represented in the linear maze within a tabular representation of the states. The rewards at state 0 should still be set at 1, and the rewards at state 9 should be set to two. The reward on all other states is 0. The discount factor should be set to gamma=0.8. 

## A Answer

In [71]:
# the network
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, optimizers

def tau(s,a):
    if (s[0] and s[9]) == 0 : s=np.roll(s,a)
    return s

def rho(s):
    return ((s[0]==1)+2*(s[9]==1))    

def terminal_state(s):
    return (s[0]==1 or s[9]==1)    

gamma=0.8
invT = 1


In [72]:
# the actual network
inputs = layers.Input(shape=(10,))
h = layers.Dense(10, activation='relu')(inputs)
outputs = layers.Dense(2, activation='linear')(h)

model = models.Model(inputs=inputs, outputs=outputs)
RMSprop = optimizers.RMSprop(lr=0.01)
model.compile(loss='mse', optimizer=RMSprop)

In [73]:
for trial in range(400):
    s= np.array([0,1,0,0,0,0,0,0,0,0])
    for t in range(0,10):
        if terminal_state(s): break
        if trial > 30 and invT > 0.1: invT -= 0.001
        prediction=model.predict(s.reshape(1,10), steps=1, verbose=0)
        aidx=np.argmax(prediction)
        if np.random.rand() < invT : aidx=1-aidx
        a=2*aidx-1
        next_s = tau(s,a)
        if terminal_state(next_s): 
            y = rho(next_s)
        else:
            y = gamma*np.max(model.predict(next_s.reshape(1,10), steps=1, verbose=0))
        prediction[0,aidx]=y
        model.fit(s.reshape(1,10), prediction, epochs=1, verbose=0)
        s = np.copy(next_s) 

In [74]:
policy = np.zeros(10)
Q=[]
s = np.array([0,0,0,1,0,0,0,0,0,0])
for i in range(0,10):
    Qs=model.predict(s.reshape(1,10), steps=1)
    Q.append(Qs)
    aidx=np.argmax(Qs)
    policy[i]=2*aidx-1
    s = np.roll(s,1)
print(np.transpose(Q))
print('policy:',np.transpose(policy))


[[[0.6739534  0.4950924  0.34532723 0.3777313  0.34155247 0.5183386
   0.48367923 0.25404304 0.97268945 0.79952955]]

 [[0.43137324 0.33400548 0.21114738 0.34472805 0.29417074 0.5062047
   0.4251778  0.24498905 0.69689345 0.5154997 ]]]
policy: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


### B.) Write a version of the program where the state input is given with a simple number representing the state.

# B Answer

In [156]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, optimizers

def tau(s,a):
    if (s == 0 and s==9) == 0 : s=s+a
    return s

def rho(s):
    return ((s==0)+2*(s==9))    

def terminal_state(s):
    return (s == 0 or s== 9)    

gamma=0.5
invT = 1

In [157]:
# the actual network
inputs = layers.Input(shape=(1,))
h = layers.Dense(10, activation='relu')(inputs)
outputs = layers.Dense(2, activation='linear')(h)

model = models.Model(inputs=inputs, outputs=outputs)
RMSprop = optimizers.RMSprop(lr=0.01)
model.compile(loss='mse', optimizer=RMSprop)

In [158]:
#Actual 

for trial in range(40):
    s=2
    for t in range(0,5):
        if terminal_state(s): break
        if trial > 30 and invT > 0.1: invT -= 0.001
        prediction=model.predict(np.array(s).reshape(1,1), steps=1, verbose=0)
        aidx=np.argmax(prediction)
        if np.random.rand() < invT : aidx=1-aidx
        a=2*aidx-1
        next_s = tau(s,a)
        if terminal_state(next_s): 
            y = rho(next_s)
        else:
            y = gamma*np.max(model.predict(np.array(next_s).reshape(1,1), steps=1, verbose=0))
        prediction[0,aidx]=y
        model.fit(np.array(s).reshape(1,1), prediction, epochs=1, verbose=0)
        s = next_s 

In [162]:
policy = np.zeros(10)
Q=[]
s = 3
for i in range(0,10):
    Qs=model.predict(np.array(s).reshape(1,1), steps=1)
    Q.append(Qs)
    aidx=np.argmax(Qs)
    policy[i]=2*aidx-1
    if s==9:
        s=0
    else:
        s=s+1
    
    
print(np.transpose(Q))
print('policy:',np.transpose(policy))


[[[0.4948008  0.4699813  0.44516206 0.42034286 0.3955235  0.37070435
   0.34588474 0.36149913 0.5370765  0.51962   ]]

 [[0.23395534 0.2220272  0.21009894 0.19817103 0.1862429  0.17431463
   0.16238661 0.1708066  0.25658476 0.24588342]]]
policy: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


### c.) Write a version of the program where a trained network is used to recognise a hand-written number from MNIST that then becomes the input.

# C Answer

In [122]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, optimizers

def tau(s,a):
    if (s == 0 and s==9) == 0 : s=s+a
    return s

def rho(s):
    return ((s==0)+2*(s==9))    

def terminal_state(s):
    return (s == 0 or s== 9)    

gamma=0.5
invT = 1

In [94]:
from keras import models,layers,optimizers,datasets,utils
(xtrain,ytrain),(xtest,ytest)= datasets.mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


In [95]:
xtrain=xtrain.reshape(60000,784)/255
xtest=xtest.reshape(10000,784)/255
ytrain=utils.to_categorical(ytrain,10)
ytest=utils.to_categorical(ytest,10)

# Giving Mnsit model ouput to agent or rnn model

In [168]:
inputs = layers.Input(shape=(784,))
x = layers.Dense(128,activation='relu')(inputs)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
outputs=layers.Dense(10,activation='softmax')(x)
# element = np.argmax(outputs)
# new_inputs=np.zeroes(10)
# new_inputs[element]=1
# inputs = layers.Input(shape=(1,))
h = layers.Dense(10, activation='relu')(outputs)
outputs = layers.Dense(2, activation='linear')(h)

model = models.Model(inputs=inputs, outputs=outputs)
RMSprop = optimizers.RMSprop(lr=0.01)
model.compile(loss='mse', optimizer=RMSprop)


In [169]:
z= [np.argmax(x) for x in  ytrain[:20]]
print(z)
image_to_y_train_value={}
y_train_value_to_image={}
for i in range(0,10):
    element_index=z.index(i)
    y_train_value_to_image[i]=xtrain[element_index]

[5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9]


In [170]:
for trial in range(50):
    s= y_train_value_to_image[2]
    s_value=2
    for t in range(0,10):
        if terminal_state(s_value): break
        if trial > 30 and invT > 0.1: invT -= 0.001
        prediction=model.predict(s.reshape(1,784), steps=1, verbose=0)
        aidx=np.argmax(prediction)
        if np.random.rand() < invT : aidx=1-aidx
        a=2*aidx-1
        next_s = tau(s_value,a)
        if terminal_state(next_s): 
            y = rho(next_s)
        else:
            next_s_image=y_train_value_to_image[next_s]
            y = gamma*np.max(model.predict(next_s_image.reshape(1,784), steps=1, verbose=0))
        prediction[0,aidx]=y
        model.fit(s.reshape(1,784), prediction, epochs=1, verbose=0)
        s = np.copy(next_s_image) 
        s_value=next_s

In [179]:
policy = np.zeros(10)
Q=[]
s = y_train_value_to_image[1]
s_value =1
for i in range(0,10):
    Qs=model.predict(s.reshape(1,784), steps=1,verbose=0)
    Q.append(Qs)
    aidx=np.argmax(Qs)
    policy[i]=2*aidx-1
    if s_value==9:
        s_value=0
    else:
        s_value+=1
    s=np.copy(y_train_value_to_image[s_value])
print(np.transpose(Q))
print('policy:',np.transpose(policy))


[[[0.5307927  0.5307927  0.5307927  0.5307927  0.5307927  0.5307927
   0.5307927  0.5307927  0.5307927  0.5307927 ]]

 [[0.48381937 0.48381937 0.48381937 0.48381937 0.48381937 0.48381937
   0.48381937 0.48381937 0.48381937 0.48381937]]]
policy: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
